In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 3
MAX_LIST_LEN = 3
MAX_STEPS = 16
EPS_START = 0.9
EPS_END = 0.1
EPS_DECAY = 2000
GAMMA = 0.9
NUM_EPISODES = 100000

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = -0.01
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = -0.02
            else:
                response_token = vocab['more']
                reward = -0.01
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = 10.0
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = 0.1
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -0.1
            else:
                reward = -0.01
            self.indices = None
        else:
            reward = -1.0
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()

# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])

def equalize_nonzero_entries(probs):
    # Move tensor to CPU if it's on CUDA for compatibility
    probs = probs.cpu() if probs.is_cuda else probs
    
    # Find the non-zero entries
    non_zero_mask = probs != 0
    
    # Count the number of non-zero entries
    non_zero_count = non_zero_mask.sum().item()
    
    if non_zero_count == 0:
        raise ValueError("Tensor has no non-zero entries to equalize.")
    
    # Set each non-zero entry to an equal value that sums to 1
    equal_value = 1.0 / non_zero_count
    probs[non_zero_mask] = equal_value
    
    # Return the modified tensor, moving it back to CUDA if needed
    return probs.to(device)


# Training Loop
def train(verbose=False):
    torch.autograd.set_detect_anomaly(True)  # Detect anomalies in autograd
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4) 

    for episode in range(NUM_EPISODES):
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        state = 'expect_action'
        done = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            output = model(input_seq)  # (seq_len, batch_size, vocab_size)
            # Get logits for the last token
            logits = output[-1, 0, :]  # (vocab_size)

            # Check for NaNs in logits
            if torch.isnan(logits).any():
                print(f"Episode {episode}, NaNs in logits before masking.")
                break

            # Get valid tokens based on state
            def get_valid_tokens(state):
                action_tokens = [vocab['Comparison'], vocab['Swap']]
                index_tokens = [vocab[str(i)] for i in range(env.length)]
                if state == 'expect_action':
                    return action_tokens
                elif state == 'expect_index1':
                    return index_tokens
                elif state == 'expect_index2':
                    return index_tokens
                else:
                    # Handle unexpected states by defaulting to expect_action
                    return action_tokens

            valid_token_ids = get_valid_tokens(state)

            # Ensure valid_token_ids are within the vocab range
            if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                break

            # Mask invalid tokens
            mask_value = -1e9  # Use a large negative value instead of -inf
            mask = torch.full_like(logits, mask_value).to(device)
            mask[valid_token_ids] = 0
            masked_logits = logits + mask

            # Check for NaNs in masked_logits
            if torch.isnan(masked_logits).any():
                print(f"Episode {episode}, NaNs in masked_logits after masking.")
                break

            # Compute probabilities
            probs = F.softmax(masked_logits, dim=0)

            # Check for NaNs in probs
            if torch.isnan(probs).any():
                print(f"Episode {episode}, NaNs in probs after softmax.")
                break

            # Sample action. Have some chance to randomly pick a valid action.
            eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
            if random.random() < eps_threshold:
                probs = equalize_nonzero_entries(probs)
            try:
                m = torch.distributions.Categorical(probs)
                action_token = m.sample()
                log_prob = m.log_prob(action_token)
            except ValueError as e:
                print(f"Episode {episode}, error in sampling action: {e}")
                break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        # Can't perform Swap without a previous Comparison
                        reward = -1.0
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if verbose:
                        print("Reward:", reward)
                    rewards.append(reward)
                    state = 'expect_action'
                else:
                    # Invalid action, end the episode
                    reward = -1.0
                    rewards.append(reward)
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if verbose:
                    print("Reward:", reward)
                rewards.append(reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                # Invalid state, end the episode
                reward = -1.0
                rewards.append(reward)
                done = True

        print(decode(input_tokens, inv_vocab))
        print(env.get_list())
                
        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns
        returns = []
        R = 0
        gamma = GAMMA
        for r in rewards[::-1]:
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)

        # Check for NaNs in returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in returns.")
            continue

        if returns.std() != 0 and not torch.isnan(returns.std()):
            returns = (returns - returns.mean()) / (returns.std() + 1e-9)
        else:
            pass #returns = returns - returns.mean()

        # Check for NaNs in standardized returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in standardized returns.")
            continue

        # Compute loss
        loss = 0
        for log_prob, R in zip(log_probs, returns):
            loss -= log_prob * R

        # Check for NaNs in loss
        if torch.isnan(loss):
            print(f"Episode {episode}, NaN in loss.")
            continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        if episode % 1 == 0:
            print(f"Episode {episode}, loss: {loss.item():.4f}, total reward: {sum(rewards):.4f}")

if __name__ == "__main__":
    train(verbose=False)


Using device: cuda


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


len3 Comparison 2 2 equal Swap Comparison 0 2 more Swap
[12, 89, 96]
Episode 0, loss: -0.0312, total reward: 9.9600
len3 Swap
[88, 93, 4]
Episode 1, loss: -0.6931, total reward: -1.0000
len3 Swap
[88, 24, 28]
Episode 2, loss: -0.6931, total reward: -1.0000
len3 Comparison 2 1 more Swap Swap
[4, 83, 2]
Episode 3, loss: -0.4682, total reward: -1.1100
len3 Comparison 0 0 equal Comparison 1 0 more Swap Comparison 1 1 equal Comparison 0
[81, 76, 96]
Episode 4, loss: -0.3639, total reward: -0.1500
len3 Comparison 1 2 more Swap
[22, 75, 79]
Episode 5, loss: 0.2867, total reward: 9.9900
len3 Swap
[36, 33, 41]
Episode 6, loss: -0.6931, total reward: -1.0000
len3 Swap
[62, 39, 46]
Episode 7, loss: -0.6931, total reward: -1.0000
len3 Swap
[61, 54, 96]
Episode 8, loss: -0.6931, total reward: -1.0000
len3 Swap
[40, 66, 70]
Episode 9, loss: -0.6931, total reward: -1.0000
len3 Comparison 2 0 less Swap Swap
[37, 33, 60]
Episode 10, loss: -0.3280, total reward: -0.9100
len3 Comparison 1 2 less Swap Com

len3 Comparison 1 0 less Swap
[10, 41, 96]
Episode 82, loss: 0.2867, total reward: 9.9900
len3 Comparison 0 0 equal Swap Swap
[95, 73, 46]
Episode 83, loss: -0.3886, total reward: -1.0300
len3 Comparison 0 0 equal Comparison 1 2 less Comparison 1 2 less Comparison 0 0 equal
[20, 48, 62]
Episode 84, loss: -0.5772, total reward: -0.0600
len3 Comparison 0 1 less Comparison 0 0 equal Comparison 1 1 equal Comparison 0 1 less
[23, 83, 80]
Episode 85, loss: -1.0399, total reward: -0.0600
len3 Comparison 1 1 equal Swap Swap
[54, 69, 42]
Episode 86, loss: -0.3886, total reward: -1.0300
len3 Comparison 2 1 less Swap Comparison 2 0 more Comparison 0 2 less Comparison 2
[59, 20, 63]
Episode 87, loss: 0.0365, total reward: 0.0700
len3 Comparison 1 2 less Swap Swap
[61, 52, 22]
Episode 88, loss: -0.4682, total reward: -1.1100
len3 Swap
[28, 78, 17]
Episode 89, loss: -0.6931, total reward: -1.0000
len3 Comparison 0 2 less Comparison 2 2 equal Swap Swap
[80, 54, 85]
Episode 90, loss: 0.8712, total rew

len3 Comparison 2 2 equal Comparison 1 1 equal Swap Swap
[91, 61, 31]
Episode 157, loss: -0.7048, total reward: -1.0500
len3 Comparison 0 2 more Swap Comparison 0 1 less Swap Swap
[49, 5, 26]
Episode 158, loss: 1.0442, total reward: -1.0200
len3 Comparison 2 1 more Comparison 0 0 equal Swap
[14, 17, 71]
Episode 159, loss: 1.0885, total reward: 9.9700
len3 Comparison 1 2 more Swap Comparison 1 2 less Comparison 2 1 more Swap Swap
[10, 58, 5]
Episode 160, loss: -1.9926, total reward: -1.0300
len3 Comparison 0 2 less Comparison 2 2 equal Swap
[33, 44, 63]
Episode 161, loss: 0.3974, total reward: 9.9700
len3 Comparison 0 2 less Swap Swap
[59, 98, 19]
Episode 162, loss: -1.2679, total reward: -1.1100
len3 Swap
[85, 53, 54]
Episode 163, loss: -0.6931, total reward: -1.0000
len3 Comparison 0 0 equal Comparison 2 0 less Swap
[39, 52, 52]
Episode 164, loss: 1.0804, total reward: 9.9700
len3 Swap
[22, 98, 23]
Episode 165, loss: -0.6931, total reward: -1.0000
len3 Swap
[96, 47, 34]
Episode 166, l

...

len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 0 2 less Comparison 2 2 equal
[19, 39, 69]
Episode 18228, loss: -0.9039, total reward: -0.0700
len3 Comparison 2 0 less Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[71, 89, 51]
Episode 18229, loss: 0.5218, total reward: -0.0700
len3 Swap
[56, 51, 72]
Episode 18230, loss: -0.7744, total reward: -1.0000
len3 Comparison 1 2 more Comparison 1 2 more Comparison 2 2 equal Comparison 0 2 more
[55, 26, 1]
Episode 18231, loss: -1.7080, total reward: -0.0500
len3 Swap
[10, 10, 11]
Episode 18232, loss: -0.7457, total reward: -1.0000
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 1 2 more Comparison 2 2 equal
[8, 96, 9]
Episode 18233, loss: -0.8298, total reward: -0.0700
len3 Comparison 2 2 equal Comparison 1 2 more Comparison 1 2 more Comparison 1 2 more
[34, 75, 61]
Episode 18234, loss: -0.7884, total reward: -0.0500
len3 Comparison 2 2 equal Comparison 1 2 more Comparison 2 2 equal Comparison 0 2 more
[57, 6

Episode 18302, loss: -0.7883, total reward: -0.0700
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 1 2 less Comparison 0 2 more
[64, 4, 56]
Episode 18303, loss: -0.8966, total reward: -0.0600
len3 Comparison 2 2 equal Comparison 2 1 more Comparison 2 2 equal Comparison 2 2 equal
[39, 35, 52]
Episode 18304, loss: -0.7704, total reward: -0.0700
len3 Comparison 1 2 less Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[16, 13, 86]
Episode 18305, loss: -2.1594, total reward: -0.0700
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 1 2 more Comparison 2 2 equal
[17, 92, 83]
Episode 18306, loss: -0.9361, total reward: -0.0700
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Comparison 2 0 less
[4, 44, 1]
Episode 18307, loss: -0.7785, total reward: -0.0700
len3 Swap
[12, 85, 65]
Episode 18308, loss: -0.7162, total reward: -1.0000
len3 Comparison 2 2 equal Comparison 0 2 more Comparison 1 2 less Comparison 2 2 equal
[92, 44, 64]
Episode 1830

len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[47, 92, 74]
Episode 18372, loss: -0.8058, total reward: -0.0800
len3 Comparison 0 2 more Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[95, 71, 94]
Episode 18373, loss: -1.3848, total reward: -0.0700
len3 Swap
[54, 30, 4]
Episode 18374, loss: -0.6447, total reward: -1.0000
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Comparison 0 2 less
[26, 18, 82]
Episode 18375, loss: -0.9270, total reward: -0.0700
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Swap Comparison 2
[92, 38, 23]
Episode 18376, loss: -0.8980, total reward: -0.0700
len3 Comparison 2 0 more Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[31, 19, 98]
Episode 18377, loss: -0.4538, total reward: -0.0700
len3 Swap
[53, 27, 16]
Episode 18378, loss: -0.6667, total reward: -1.0000
len3 Swap
[77, 17, 49]
Episode 18379, loss: -0.6486, total reward: -1.0000
len3 Swap
[7

len3 Comparison 0 2 more Comparison 2 2 equal Comparison 1 2 more Comparison 2 1 less
[47, 96, 26]
Episode 18450, loss: -2.5042, total reward: -0.0500
len3 Swap
[100, 78, 21]
Episode 18451, loss: -0.7468, total reward: -1.0000
len3 Swap
[89, 15, 46]
Episode 18452, loss: -0.8014, total reward: -1.0000
len3 Swap
[75, 82, 34]
Episode 18453, loss: -0.8159, total reward: -1.0000
len3 Swap
[10, 39, 91]
Episode 18454, loss: -0.6892, total reward: -1.0000
len3 Swap
[53, 59, 54]
Episode 18455, loss: -0.8427, total reward: -1.0000
len3 Swap
[1, 14, 94]
Episode 18456, loss: -0.7739, total reward: -1.0000
len3 Comparison 2 2 equal Swap Comparison 2 2 equal Comparison 2 2 equal Comparison 2
[95, 13, 32]
Episode 18457, loss: 0.1235, total reward: -0.0700
len3 Swap
[7, 67, 60]
Episode 18458, loss: -0.8161, total reward: -1.0000
len3 Comparison 2 2 equal Comparison 0 2 less Comparison 2 2 equal Comparison 2 2 equal
[29, 90, 63]
Episode 18459, loss: -0.7613, total reward: -0.0700
len3 Comparison 1 1 eq

Episode 18526, loss: -0.7494, total reward: -0.0700
len3 Comparison 0 2 more Swap Comparison 0 2 less Comparison 2 2 equal Comparison 2
[8, 64, 28]
Episode 18527, loss: 1.0223, total reward: 0.0600
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Comparison 0 2 more
[94, 42, 82]
Episode 18528, loss: -0.7844, total reward: -0.0700
len3 Swap
[96, 28, 18]
Episode 18529, loss: -0.7126, total reward: -1.0000
len3 Comparison 1 2 less Comparison 2 0 more Swap Comparison 2 2 equal Comparison 2
[60, 52, 7]
Episode 18530, loss: -1.4245, total reward: -0.1400
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 1 2 less Swap Comparison 2
[52, 88, 25]
Episode 18531, loss: -0.8037, total reward: -0.1500
len3 Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[67, 91, 17]
Episode 18532, loss: -0.7777, total reward: -0.0800
len3 Comparison 1 2 less Comparison 2 2 equal Comparison 2 2 equal Comparison 2 2 equal
[66, 61, 72]
Episode 18533, loss: 